<a href="https://colab.research.google.com/github/JAshinflame/AI-Agents/blob/main/method_statement_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install python-docx

In [6]:
!pip install reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.8 MB/s eta 0:00:00


In [8]:
# Method Statement Automation Pipeline
# =====================================
# This notebook parses BOQ & project details DOCX files and generates a draft Method Statement PDF.
# Run each cell sequentially.

from pathlib import Path
import docx
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, PageBreak
from reportlab.lib import colors
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm

# === Paths ===
boq_path = Path('ASB - BOQ Extract.docx')
project_path = Path('ASB Mini Project Details.docx')
template_path = Path('Test 3. Method Statement MS.docx')
output_pdf = Path('generated_method_statement.pdf')

# === Helpers ===
def extract_tables(docx_path):
    doc = docx.Document(docx_path)
    tables = []
    for tbl in doc.tables:
        rows = [[c.text.strip() for c in r.cells] for r in tbl.rows]
        tables.append(rows)
    return tables

def parse_boq(boq_path):
    tables = extract_tables(boq_path)
    boq_rows = None
    for tbl in tables:
        header = ' '.join(tbl[0]).lower()
        if 'item' in header and 'description' in header:
            boq_rows = tbl
            break
    if boq_rows is None and tables:
        boq_rows = tables[0]

    items = []
    if boq_rows:
        headers = [h.strip() for h in boq_rows[0]]
        for r in boq_rows[1:]:
            if all([c == '' for c in r]):
                continue
            row = {headers[i]: r[i] if i < len(r) else '' for i in range(len(headers))}
            items.append(row)
    return items

def parse_project_info(project_path):
    tables = extract_tables(project_path)
    info = {}
    for tbl in tables:
        for row in tbl:
            if len(row) >= 2:
                k, v = row[0].strip().rstrip(':'), row[1].strip()
                if k: info[k] = v
    doc = docx.Document(project_path)
    for p in doc.paragraphs:
        text = p.text.strip()
        if ':' in text and len(text.split(':')[0]) < 40:
            k, v = text.split(':', 1)
            info.setdefault(k.strip(), v.strip())
    return info

# === Main ===
boq_items = parse_boq(boq_path)
project_info = parse_project_info(project_path)

project_name = project_info.get('Project Name')
location = project_info.get('Location')
client = project_info.get('Client')
main_contractor = project_info.get('Main Contractor')
sub_contractor = project_info.get('Sub-Contractor')
activity = project_info.get('Activity for Method Statement Test')

relevant = [it for it in boq_items if any(k in ' '.join(it.values()).lower() for k in ['block','plaster','render','paint'])]

scope = [
    f'This Method Statement covers the execution of {activity} for the project {project_name} located at {location}.',
    'Work includes blockwork, plastering, and painting per project specifications.'
]
references = [
    'Project Drawings: A-201, A-305',
    'Technical Specifications: Section 04200 (Masonry), Section 09200 (Plastering)',
    'BOQ Extract: Provided'
]
materials = [f"{it.get('Description of Work','')} — {it.get('Quantity','')} {it.get('Unit','')}".strip() for it in relevant]
if not materials:
    materials = ['Hollow concrete blocks', 'Cement-sand mortar', 'Cement plaster', 'Paint (emulsion)']
procedure = []

doc = SimpleDocTemplate(str(output_pdf), pagesize=A4,
                        rightMargin=20*mm, leftMargin=20*mm,
                        topMargin=20*mm, bottomMargin=20*mm)
styles = getSampleStyleSheet()
styleN, styleH2 = styles['Normal'], styles['Heading2']
elements = []

elements.append(Paragraph('METHOD STATEMENT', ParagraphStyle('Title', fontSize=16, alignment=1)))
elements.append(Spacer(1,8))
meta = [['Project Name:', project_name], ['Location:', location], ['Client:', client]]
t = Table(meta, colWidths=[90*mm,80*mm])
t.setStyle(TableStyle([('BACKGROUND',(0,0),(0,-1),colors.lightgrey),('GRID',(0,0),(-1,-1),0.25,colors.grey)]))
elements += [t, Spacer(1,12)]

def add_section(title, lines):
    elements.append(Paragraph(title, styleH2))
    for ln in lines: elements.append(Paragraph(ln, styleN)); elements.append(Spacer(1,5))
    elements.append(Spacer(1,10))

add_section('Scope', scope)
add_section('References', references)
add_section('Materials', [f'• {m}' for m in materials])
add_section('Work Procedure', procedure)

doc.build(elements)
print(f'✅ Generated draft Method Statement saved as: {output_pdf.resolve()}')


✅ Generated draft Method Statement saved as: /content/generated_method_statement.pdf
